<a href="https://colab.research.google.com/github/RoseSarlake/Computer-Vision/blob/main/CV_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Harris derector

a) Assume 𝜎0 = 0.5 as the smoothing of the input image. Compute the first derivative matrixes (Ix ,Iy) at a scale 𝜎𝐷 = 1.4𝜎0 . Differential masks of size 2 can be used.

b) Compute the three matrixes: Ix2, Iy2, IxIy

c) Compute at each pixel the terms of the second moment matrix by convolving the three matrices in point (c) with an isotropic Gaussian of 𝜎𝐼 = 1.5𝜎𝐷. Compute Trace and Determinant in each pixel

d) Apply the Harris-corner criteria to compute the Harris-feature map

e) Apply Non-Maxima suppression to identify true Harris points. Use 5 pixels as the minimum distance between maxima.

f) Fix a threshold to keep only with the first 1000 highest values.

g) Refine the spatial coordinates of each KeyPoint using OpenCV's function cornerSubPix on a window of size 5x5.

# Orientation and Display

h) Extract a signature vector, 𝑆𝑉 = (𝑥, 𝑦, 𝜃, 𝜆1, 𝜆2, 𝜎𝐼), for each Harris-point. Compute the 
region orientation 𝜃 using the angle between the eigenvector associated to the highest eigenvalue (the semi-major axis of the ellipse) and the axis x of the image

i) Show all detected points drawing on the original image circles of radius proportional to the detection scale 𝜎𝐼 with an radio point out to the direction of 𝜃 (Hint: compute KeyPoints and use drawKeyPoints ()).

j) Show again all detected points but now drawing ellipses instead of circles and using for it the information the first five values of the signature vector. (Hint: use ellipse())

# Region rectification

k) Compute the isometric rectified local regions associated with Harris point integration regions. Use Baumberg’s algorithm and the Cholesky decomposition. Normalize the intensity graylevel to || ||F=1 (Frobenius’s norm)

l) Show in a panel with at least 10 original and rectified regions. As the integrations regions will be too small apply a 10x interpolation to increase the 
size.

# BONUS

m) Construct a four-level Gaussian Scale-space. Assume 𝜎0 = 0.5 𝑎𝑛𝑑 𝜎𝑘 = 1.4 × 𝜎𝑘−1, 𝑘 = 1,2,3,4, as smoothing sigma on each scale. These sigma values fix 
the scale 𝜎𝐷 of the derivative on each level. Apply the above process to all pyramid levels selecting the most representatives regions from each level to a total of 1500. Show the result.